In [82]:
import pandas as pd
import numpy as np
import math

In [83]:
RANDOM_SEED = 13022022

In [84]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [85]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  1614 non-null   object
dtypes: object(1)
memory usage: 12.7+ KB


In [86]:
test.sample(3)

,Smiles
826,CC(N)Cc1cnc[nH]1
452,COc1ccc2c(c1)c(C1=C(c3c[nH]c4ccccc34)C(=O)NC1=...
1574,C=CCN1CCc2nc(N)sc2CC1.Cl


In [87]:
train = pd.read_csv('Task/train.csv')
train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 49.0+ KB


In [89]:
train.sample(3)

,Smiles,Active
2975,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,False
1706,CC(C)c1ccc(NC(=O)c2cccs2)c(-c2nc3ccccc3[nH]c2=...,False
1793,Cc1cc(O)c(C(C)C)cc1Cl,False


In [90]:
list(train['Active'].unique())

[False, True]

In [91]:
list_active = {False: 0, True: 1}

In [92]:
train['Active'] = train['Active'].map(list_active)

In [93]:
train.sample()

,Smiles,Active
2738,O=C(O)c1cc(Oc2ccc(C(F)(F)F)cc2Cl)ccc1[N+](=O)[O-],0


In [94]:
train['train'] = 1
test['train'] = 0

In [95]:
df = pd.concat([train, test], ignore_index=True)

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7171 entries, 0 to 7170
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Smiles  7171 non-null   object 
 1   Active  5557 non-null   float64
 2   train   7171 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 168.2+ KB


In [97]:
df['train'].unique()

array([1, 0], dtype=int64)

In [98]:
df[df['train'] == 1]['Active'].value_counts()

0.0    5351
1.0     206
Name: Active, dtype: int64

In [99]:
from collections import Counter

atoms = Counter()
for i in df['Smiles'].unique():
    for j in i:
        atoms[j] += 1

In [100]:
len(atoms.keys())

45

In [101]:
cont = 0
for i in atoms.keys():
    new_value = len(atoms.keys()) - cont
    atoms[i] = new_value
    cont += 1

In [102]:
atoms

Counter({'C': 45,
         'O': 44,
         'c': 43,
         '1': 42,
         '2': 41,
         '[': 40,
         'n': 39,
         'H': 38,
         ']': 37,
         '(': 36,
         'N': 35,
         ')': 34,
         '@': 33,
         '.': 32,
         'l': 31,
         '=': 30,
         'S': 29,
         '3': 28,
         '4': 27,
         'F': 26,
         '-': 25,
         's': 24,
         '/': 23,
         '5': 22,
         'o': 21,
         'a': 20,
         '+': 19,
         '#': 18,
         'I': 17,
         'P': 16,
         'B': 15,
         'r': 14,
         '\\': 13,
         'Z': 12,
         '6': 11,
         '7': 10,
         '8': 9,
         'e': 8,
         'A': 7,
         'K': 6,
         'M': 5,
         'g': 4,
         'i': 3,
         'L': 2,
         '9': 1})

In [103]:
df['1'] = df['Smiles'].apply(lambda x: list(x))

In [112]:
df['len'] = df['Smiles'].apply(lambda x: len(x))

In [113]:
df['len'].unique().max()

707

In [117]:
new_df = pd.DataFrame(columns = range(0,812))

In [119]:
new_df.shape

(0, 812)

In [120]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,802,803,804,805,806,807,808,809,810,811


In [121]:
e = pd.DataFrame(df.loc[0].explode())

In [122]:
e = e.T

In [123]:
new_df = new_df.append(e)

ValueError: Plan shapes are not aligned

In [107]:
w = pd.DataFrame(df.loc[1].explode())

In [108]:
w = w.T

In [109]:
w

,Smiles,Active,train,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,CCCN1CCC[C@H](c2cccc(O)c2)C1.Cl,0,1,C,C,C,N,1,C,C,...,O,),c,2,),C,1,.,C,l


In [111]:
e = e.append(w, ignore_index=True)

ValueError: Plan shapes are not aligned

In [80]:
def count_i(x):
    atoms = Counter()
    for i in x:
        atoms[i] += 1
    return atoms

In [88]:
train_new = df[df['train'] == 1]
test_new = df[df['train'] == 0]

In [90]:
X = train_new.drop(['Active'], axis = 1)
y = train_new['Active']

In [91]:
from sklearn import model_selection, datasets, metrics, tree 

In [92]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, 
                                                                                    test_size=0.30, random_state=RANDOM_SEED)

In [107]:
from lazypredict.Supervised import LazyClassifier

In [108]:
clf_lc = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf_lc.fit(train_data, test_data, train_labels, test_labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:21<00:00,  1.37it/s]


In [109]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
DecisionTreeClassifier             0.93               0.55     0.55      0.93   
NearestCentroid                    0.75               0.54     0.54      0.83   
ExtraTreeClassifier                0.93               0.53     0.53      0.93   
LabelSpreading                     0.94               0.52     0.52      0.93   
LabelPropagation                   0.94               0.52     0.52      0.93   
XGBClassifier                      0.96               0.52     0.52      0.94   
Perceptron                         0.94               0.51     0.51      0.93   
BaggingClassifier                  0.95               0.51     0.51      0.94   
SGDClassifier                      0.95               0.51     0.51      0.94   
LinearDiscriminantAnalysis         0.95               0.51     0.51      0.94   
QuadraticDiscriminantAnalysi

In [99]:
from sklearn.neighbors import NearestCentroid

In [100]:
NC_model = NearestCentroid()

In [101]:
train_data, test_data, train_labels, test_labels
NC_model.fit(train_data, train_labels)
predictions_clf_tree = NC_model.predict(test_data)
metrics.f1_score(test_labels, predictions_clf_tree)

0.10037174721189593

In [102]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()


In [103]:
clf_rf.fit(train_data, train_labels)
predictions_clf_rf = clf_rf.predict(test_data)
metrics.f1_score(test_labels, predictions_clf_rf)

0.0273972602739726

In [103]:
test_new_data = test_new.drop(['Active'], axis = 1) 

In [105]:
predictions = clf.predict(test_new_data)

In [106]:
test_new_data['Active'] = predictions

In [107]:
test_new_data.sample(5)

,train,len_smiles,C,O,c,1,2,[,n,H,...,8,e,A,K,M,g,i,L,9,Active
Smiles,,,,,,,,,,,,,,,,,,,,,
Nc1c2c(nc3ccccc13)CCCC2O.O=C(O)/C=C\C(=O)O,0,42,8,5,9,2,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
CN(C)CCCN1c2ccccc2Sc2ccc(C(F)(F)F)cc21.Cl,0,41,7,0,12,2,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
CCC(NC(C)C)C(O)c1ccc(O)c2[nH]c(=O)ccc12,0,39,7,3,9,2,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0.0
Cn1c(=O)cnn(CCCCN2CCN(c3ncccn3)CC2)c1=O,0,39,9,2,7,2,2,0,5,0,...,0,0,0,0,0,0,0,0,0,0.0
COc1ccc(O)c(C(C)(C)C)c1,0,23,5,2,6,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [108]:
test_new_data.reset_index(inplace = True)

In [109]:
test_new_data.sample(5)

,Smiles,train,len_smiles,C,O,c,1,2,[,n,...,8,e,A,K,M,g,i,L,9,Active
279,CC[C@H](CO)Nc1nc(Nc2cccc(-c3ccccn3)c2)c2ncn(C(...,0,54,7,1,16,2,4,1,5,...,0,0,0,0,0,0,0,0,0,1.0
1189,CCC(=O)O[C@H]1[C@H](O[C@@H]2[C@@H](C)[C@H](O[C...,0,206,52,18,0,2,2,18,0,...,0,0,0,0,0,0,0,0,0,0.0
1517,COc1ccc(-c2c[nH]c3nccc(-c4ccsc4)c23)c(OC)c1,0,43,2,2,17,2,2,1,2,...,0,0,0,0,0,0,0,0,0,0.0
280,CNC(=O)c1cc2c(cn1)[nH]c1ccccc12,0,31,2,1,11,4,2,1,2,...,0,0,0,0,0,0,0,0,0,0.0
60,CN(C)C(=O)Oc1cc(OC(=O)N(C)C)cc(C(O)CNC(C)(C)C)c1,0,48,12,5,6,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [110]:
submission = test_new_data[['Smiles', 'Active']]

In [111]:
submission.sample(4)

,Smiles,Active
1504,Oc1ccc2[nH]c(-c3ccccc3)nc2c1CN1CCCCC1,0.0
1069,CCN(Cc1ccc(Cl)nc1)/C(=C/[N+](=O)[O-])NC,0.0
396,C[C@@H]1[C@H]2C3=CC[C@@H]4[C@@]5(C)CC[C@H](O)C...,0.0
41,COc1ccnc(C[S+]([O-])c2nc3cc(OC(F)F)ccc3[nH]2)c1OC,1.0


In [112]:
submission['Active'] = submission['Active'].apply(lambda x: int(x))

C:\Users\Rusla\AppData\Local\Temp/ipykernel_4936/2085481574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['Active'] = submission['Active'].apply(lambda x: int(x))


In [113]:
submission.to_csv('submission.csv', index=False)